## TODO
 - Make a list of important dates (holidays, fast sundays, stake conferences)
 - Add a check for all future-scheduled dates that they don't have any hymns that were sung recently. This would happen if a hymn got changed after the schedule was made.

# Steps to use:
 1. Make sure the history is accurate for the past few weeks
 2. Check if there are any holidays, fast sundays, or special meetings coming up
 3. Run most of the notebook to generate suggestions
 4. Select hymns, enter on Edify
 5. Add selected hymns to history
 6. Add hymns to Gospel Library
 7. Update ward program, if needed, using Gospel Library as reference


In [1]:
import datetime as dt

import numpy as np
import polars as pl

pl.Config().set_tbl_rows(30)
pl.Config().set_fmt_str_lengths(50)

polars.config.Config

In [2]:
def load_history() -> pl.DataFrame:
    history_df = pl.read_csv('history.csv', infer_schema_length=None)
    history_df = history_df.with_columns(
        pl.col("date").str.strptime(pl.Date, "%m/%d/%Y").alias("date"),
    ).drop('name').drop_nulls()
    return history_df

def error_check_history() -> None:
    """Check that no future dates are duplicates with any dates since 8 weeks ago."""
    history_df = load_history()
    num_weeks_cutoff = 8
    today = dt.datetime.today().date()
    lookback_date = today - dt.timedelta(days=7 * num_weeks_cutoff)
    recent_and_future_history = history_df.filter(pl.col("date") >= lookback_date)
    value_counts = recent_and_future_history["id"].value_counts()
    recent_and_future_history = recent_and_future_history.join(
        value_counts, on="id", how="left"
    ).rename({"count": "recent_count"})

    future_history = recent_and_future_history.filter(pl.col("date") >= today)
    duplicates = future_history.filter(pl.col("recent_count") > 1)
    if len(duplicates) > 0:
        display(duplicates)
        print(">>> WARNING! DUPLICATES FOUND! <<<\n")

def add_last_sung_col(df):
    history_df = load_history()
    last_sung = history_df.group_by('id').agg(pl.max('date').alias('last_sung'))
    df = df.join(last_sung, on='id', how='left')
    return df


def get_ranking_score(df: pl.DataFrame, type_col: str, popularity_weight=0.5, noise=0.0) -> pl.Series:
    score_df = df.filter((pl.col(type_col) == 1) & (pl.col('flagged').fill_null(0) != 1))
    weeks_since_fill_value = 100
    # Add days since last sung column
    score_df = score_df.with_columns(
        ((pl.lit(dt.datetime.today()) - pl.col('last_sung')).dt.total_days()//7)
        .fill_null(weeks_since_fill_value)
        .alias('weeks_since')
    )

    midpoint = 7
    slope = 2

    def popularity_func(x): return np.round(
        1 / (1 + np.exp(-slope * (x - midpoint))), 2)
    weeks_cutoff = 8
    score_df = score_df.with_columns((pl.col('popularity') + pl.col('popularity_adjustment').fill_null(0)).alias('adj_popularity'))
    score_df = score_df.with_columns(
        pl.col('adj_popularity').map_elements(popularity_func).alias('popularity_score'),
        pl.when(pl.col('weeks_since') > weeks_cutoff)
        .then((pl.col('weeks_since') / (weeks_since_fill_value/2)).round(3))
        .otherwise(-100).alias('weeks_since_score'),
        pl.Series(np.round(np.random.uniform(-noise, noise,
                  len(score_df)), 2)).alias('noise'),
    ).with_columns(
        (pl.col('popularity_score') * popularity_weight + pl.col('weeks_since_score')
         * (1 - popularity_weight)).alias('score') + pl.col('noise')
    )

    score_df = score_df.select([
        'id', 'name', 'length', 'adj_popularity', 'weeks_since', 'popularity_score', 'weeks_since_score', 'score'
    ]).sort('score', descending=True)

    return score_df

In [3]:
error_check_history()
df = pl.read_csv("hymns.csv")
df = add_last_sung_col(df)

## Sacrament Hymn

In [4]:
sacrament_ranking = get_ranking_score(df, 'is_sacrament', noise=0.1)
display(sacrament_ranking.head(10))

id,name,length,adj_popularity,weeks_since,popularity_score,weeks_since_score,score
i64,str,str,i64,i64,f64,f64,f64
146,"""Gently Raise the Sacred Strain""","""3:55""",8,33,0.88,0.66,0.84
171,"""With Humble Heart""","""2:44""",10,22,1.0,0.44,0.82
188,"""Thy Will, O Lord, Be Done""","""2:18""",10,29,1.0,0.58,0.81
177,"""Tis Sweet to Sing the Matchless Love (HANCOCK)""","""3:08""",10,25,1.0,0.5,0.79
180,"""Father in Heaven, We Do Believe""","""2:14""",10,19,1.0,0.38,0.73
194,"""There Is a Green Hill Far Away""","""2:19""",10,24,1.0,0.48,0.71
193,"""I Stand All Amazed""","""3:41""",10,17,1.0,0.34,0.7
186,"""Again We Meet around the Board""","""2:57""",8,16,0.88,0.32,0.68
169,"""As Now We Take the Sacrament""","""3:01""",10,15,1.0,0.3,0.66


## General Hymns

In [5]:
general_ranking = get_ranking_score(df, 'is_general', noise=0.15, popularity_weight=0.7)
display(general_ranking.head(30))

id,name,length,adj_popularity,weeks_since,popularity_score,weeks_since_score,score
i64,str,str,i64,i64,f64,f64,f64
219,"""Because I Have Been Given Much""",null,10,24,1.0,0.48,0.954
131,"""More Holiness Give Me""","""2:49""",9,25,0.98,0.5,0.946
113,"""Our Savior's Love""","""2:55""",9,23,0.98,0.46,0.944
86,"""How Great Thou Art""",null,8,41,0.88,0.82,0.942
134,"""I Believe in Christ""","""5:00""",8,32,0.88,0.64,0.908
223,"""Have I Done Any Good?""","""2:46""",8,31,0.88,0.62,0.902
304,"""Teach Me to Walk in the Light""","""2:18""",10,12,1.0,0.24,0.842
133,"""Father in Heaven""","""2:41""",6,100,0.12,2.0,0.834
6,"""Redeemer of Israel""","""2:45""",9,18,0.98,0.36,0.834


# Add to History

In [6]:
raise KeyboardInterrupt("This is here to stop you from running the whole notebook.")

KeyboardInterrupt: This is here to stop you from running the whole notebook.

In [ ]:
def name_from_id(id):
    name_row = df.filter(pl.col("id") == id).select("name")
    if len(name_row) != 1:
        raise ValueError(f"Expected 1 row, got {len(name_row)}")
    return name_row["name"].item()


def format_for_csv(date, opening, sacrament, intermediate, closing):
    str_list = []
    if opening:
        str_list.append(
            f"{date.strftime('%m/%d/%Y')},opening,{opening},\"{name_from_id(opening)}\""
        )
    if sacrament:
        str_list.append(
            f"{date.strftime('%m/%d/%Y')},sacrament,{sacrament},\"{name_from_id(sacrament)}\""
        )
    if intermediate:
        str_list.append(
            f"{date.strftime('%m/%d/%Y')},intermediate,{intermediate},\"{name_from_id(intermediate)}\""
        )
    if closing:
        str_list.append(
            f"{date.strftime('%m/%d/%Y')},closing,{closing},\"{name_from_id(closing)}\""
        )
    csv_str = "\n".join(str_list)
    return csv_str

date = dt.date(2024, 3, 20) # EDIT ME!
opening = 89
sacrament = 128
intermediate = 134
closing = 123

In [ ]:
csv_str = format_for_csv(date, opening, sacrament, intermediate, closing)
print("Does this look right?")
print(csv_str)

Does this look right?
03/20/2024,opening,89,"The Lord Is My Light"
03/20/2024,sacrament,128,"When Faith Endures"
03/20/2024,intermediate,134,"I Believe in Christ"
03/20/2024,closing,123,"Oh, May My Soul Commune with Thee"


In [ ]:
# Append to history csv
with open('history.csv', 'a') as f:
    f.write(csv_str)
    f.write("\n")